First we can load the file containting the winnig side for each game id
and can print some basic info like total number of games and winrate per side

In [182]:
import pandas as pd
wins = pd.read_csv('csv/sql/wins.csv')
display(wins.head())
blue_wins = wins[wins["winningteam"] == 100]
red_wins = wins[wins["winningteam"] == 200]
print("Total wins: \t", len(wins))
print("Blue wins: \t", len(blue_wins))
print("Red wins: \t", len(red_wins))
print("Blue win rate: \t", len(blue_wins)/len(wins))
print("Red win rate: \t", len(red_wins)/len(wins))

,platformgameid,winningteam
0,ESPORTSTMNT01:1294233,100
1,ESPORTSTMNT01:1155159,100
2,ESPORTSTMNT01:1144593,200
3,ESPORTSTMNT01:1435288,200
4,ESPORTSTMNT01:1293312,100


Total wins: 	 25255
Blue wins: 	 13352
Red wins: 	 11903
Blue win rate: 	 0.5286873886359137
Red win rate: 	 0.4713126113640863


First, we load all the data for first blood, first_woter and and first inhib.
the data for tower and in inhibv are contained within one and.

In [183]:
team_games = pd.read_csv("csv/sql/team_games.csv", dtype= {'teamid': 'str'})
team_games["teamid"].fillna(0, inplace=True)
team_games.sort_values(by=['platformgameid'], inplace=True)

blue_side = team_games[team_games["side"] == 100]
blue_side = blue_side.rename(columns={
    "teamid": "blue_teamid",
})
blue_side.drop(columns=["side"], inplace=True)

red_side = team_games[team_games["side"] == 200]
red_side = red_side.rename(columns={
    "teamid": "red_teamid",
})
red_side.drop(columns=["side", "esportsgameid"], inplace=True)

joined_side = blue_side.merge(red_side, on="platformgameid")
wins = wins.merge(joined_side, on="platformgameid")
display(wins.head())

,platformgameid,winningteam,blue_teamid,esportsgameid,red_teamid
0,ESPORTSTMNT01:1294233,100,100285330168091787,103540398659213461,101428372602011186
1,ESPORTSTMNT01:1155159,100,99566404579461230,102181142524096199,100205573496804586
2,ESPORTSTMNT01:1144593,200,99566405944694628,102147203790063742,99566408358330419
3,ESPORTSTMNT01:1435288,200,99566408217955692,104202471517880208,103743162394217975
4,ESPORTSTMNT01:1293312,100,100725845022060229,103540364360693654,102747101562758215


In [184]:
first_blood = pd.read_csv("csv/sql/first_blood.csv")
first_blood = first_blood.sort_values(by=["platformgameid", "gametime"])
first_blood = first_blood.drop_duplicates(subset=['platformgameid'], keep='first')
first_blood  = first_blood[["platformgameid", "gametime", "killerteamid"]]
first_blood = first_blood.rename(columns={
    'killerteamid': 'first_blood_team_id',
    "gametime": "first_blood_time",
})

print("First blood: \t", len(first_blood))
display(first_blood.head())

first_building = pd.read_csv("csv/sql/first_buildings.csv")
first_building  = first_building[["platformgameid", "gametime", "teamid", "buildingtype"]]

first_tower = first_building[first_building["buildingtype"] == "turret"].copy()
first_tower = first_tower.sort_values(by=["platformgameid", "gametime"])
first_tower = first_tower.drop_duplicates(subset=['platformgameid'], keep='first')
# killer_team_id = first_tower["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_tower["first_tower_owner_id"] = first_tower["teamid"]
first_tower = first_tower.drop(columns=["teamid", "buildingtype"])
first_tower = first_tower.rename(columns={
    "gametime": "first_tower_time",
})
print("First tower: \t", len(first_tower))
display(first_tower.head())

first_inhibitor = first_building[first_building["buildingtype"] == "inhibitor"].copy()
first_inhibitor = first_inhibitor.sort_values(by=["platformgameid", "gametime"])
first_inhibitor = first_inhibitor.drop_duplicates(subset=['platformgameid'], keep='first')

first_inhibitor["first_inhibitor_owner_id"] = first_inhibitor["teamid"]
first_inhibitor = first_inhibitor.drop(columns=["teamid", "buildingtype"])
first_inhibitor = first_inhibitor.rename(columns={
    "gametime": "first_inhibitor_time",
})
print("First inhibitor: \t", len(first_inhibitor))
first_inhibitor.head()

First blood: 	 25250


,platformgameid,first_blood_time,first_blood_team_id
4535,ESPORTSTMNT01:1110148,245612,100
9142,ESPORTSTMNT01:1110162,192537,100
21281,ESPORTSTMNT01:1110177,207700,100
35484,ESPORTSTMNT01:1110187,359245,200
11323,ESPORTSTMNT01:1110198,148483,100


First tower: 	 25252


,platformgameid,first_tower_time,first_tower_owner_id
40575,ESPORTSTMNT01:1110148,939441,200
23132,ESPORTSTMNT01:1110162,677086,200
31938,ESPORTSTMNT01:1110177,836608,200
12714,ESPORTSTMNT01:1110187,626524,100
39594,ESPORTSTMNT01:1110198,895424,200


First inhibitor: 	 25212


,platformgameid,first_inhibitor_time,first_inhibitor_owner_id
70824,ESPORTSTMNT01:1110148,1708748,200
41143,ESPORTSTMNT01:1110162,1602302,200
62898,ESPORTSTMNT01:1110177,1546286,200
3623,ESPORTSTMNT01:1110187,1628728,200
50722,ESPORTSTMNT01:1110198,1694081,200


In [185]:
df = pd.merge(first_blood, first_tower, on="platformgameid", how="outer")
df = df.merge(first_inhibitor, on="platformgameid", how="outer")
df[["platformgameid", "first_blood_team_id", "first_tower_owner_id", "first_inhibitor_owner_id"]]

,platformgameid,first_blood_team_id,first_tower_owner_id,first_inhibitor_owner_id
0,ESPORTSTMNT01:1110148,100.0,200,200.0
1,ESPORTSTMNT01:1110162,100.0,200,200.0
2,ESPORTSTMNT01:1110177,100.0,200,200.0
3,ESPORTSTMNT01:1110187,200.0,100,200.0
4,ESPORTSTMNT01:1110198,100.0,200,200.0
...,...,...,...,...
25247,LPL_B:330033,100.0,100,100.0
25248,LPL_B:330039,100.0,200,100.0
25249,LPL_B:330053,100.0,200,200.0
25250,ESPORTSTMNT02:1114248,NaN,200,200.0


In [186]:
blue_fb = df[df["first_blood_team_id"] == 100]
red_fb = df[df["first_blood_team_id"] == 200]
print("Blue first blood: \t", len(blue_fb))
print("Red first blood: \t", len(red_fb))
print("\n")
blue_fb_tower = df[df["first_tower_owner_id"] == 200]
red_fb_tower = df[df["first_tower_owner_id"] == 100]
print("Blue first tower: \t", len(blue_fb_tower))
print("Red first tower: \t", len(red_fb_tower))
blue_fb_inhibitor = df[df["first_inhibitor_owner_id"] == 200]
red_fb_inhibitor = df[df["first_inhibitor_owner_id"] == 100]
print("Blue first inhibitor: \t", len(blue_fb_inhibitor))
print("Red first inhibitor: \t", len(red_fb_inhibitor))

Blue first blood: 	 12885
Red first blood: 	 12323


Blue first tower: 	 13875
Red first tower: 	 11377
Blue first inhibitor: 	 13409
Red first inhibitor: 	 11803


In [187]:
df = df.merge(wins, on="platformgameid", how="outer")
df.head()

,platformgameid,first_blood_time,first_blood_team_id,first_tower_time,first_tower_owner_id,first_inhibitor_time,first_inhibitor_owner_id,winningteam,blue_teamid,esportsgameid,red_teamid
0,ESPORTSTMNT01:1110148,245612.0,100.0,939441,200,1708748.0,200.0,100.0,100783238110621058,1.019782e+17,99566406336293850
1,ESPORTSTMNT01:1110162,192537.0,100.0,677086,200,1602302.0,200.0,100.0,98767991949608898,1.019782e+17,99566408358330419
2,ESPORTSTMNT01:1110177,207700.0,100.0,836608,200,1546286.0,200.0,100.0,100783238110621058,1.019782e+17,99566408358330419
3,ESPORTSTMNT01:1110187,359245.0,200.0,626524,100,1628728.0,200.0,100.0,98767991949608898,1.019782e+17,99566406336293850
4,ESPORTSTMNT01:1110198,148483.0,100.0,895424,200,1694081.0,200.0,100.0,99566406336293850,1.019782e+17,99566408358330419


In [188]:
df.head()
blue_fb_win = df[(df["first_blood_team_id"] == 100) & (df["winningteam"] == 100)]
red_fb_win = df[(df["first_blood_team_id"] == 200) & (df["winningteam"] == 200)]
print("Blue first blood win rate: \t", len(blue_fb_win)/len(blue_fb))
print("Red first blood win rate: \t", len(red_fb_win)/len(red_fb))


blue_fb_tower_win = df[(df["first_tower_owner_id"] == 200) & (df["winningteam"] == 100)]
red_fb_tower_win = df[(df["first_tower_owner_id"] == 100) & (df["winningteam"] == 200)]
print("Blue first tower win rate: \t", len(blue_fb_tower_win)/len(blue_fb_tower))
print("Red first tower win rate: \t", len(red_fb_tower_win)/len(red_fb_tower))

blue_fb_inhibitor_win = df[(df["first_inhibitor_owner_id"] == 200) & (df["winningteam"] == 100)]
red_fb_inhibitor_win = df[(df["first_inhibitor_owner_id"] == 100) & (df["winningteam"] == 200)]
print("Blue first inhibitor win rate: \t", len(blue_fb_inhibitor_win)/len(blue_fb_inhibitor))
print("Red first inhibitor win rate: \t", len(red_fb_inhibitor_win)/len(red_fb_inhibitor))

Blue first blood win rate: 	 0.6388048117966628
Red first blood win rate: 	 0.5865454840542076
Blue first tower win rate: 	 0.6958558558558559
Red first tower win rate: 	 0.6759251120682078
Blue first inhibitor win rate: 	 0.9446640316205533
Red first inhibitor win rate: 	 0.9447598068287724


In [189]:
df.head()

,platformgameid,first_blood_time,first_blood_team_id,first_tower_time,first_tower_owner_id,first_inhibitor_time,first_inhibitor_owner_id,winningteam,blue_teamid,esportsgameid,red_teamid
0,ESPORTSTMNT01:1110148,245612.0,100.0,939441,200,1708748.0,200.0,100.0,100783238110621058,1.019782e+17,99566406336293850
1,ESPORTSTMNT01:1110162,192537.0,100.0,677086,200,1602302.0,200.0,100.0,98767991949608898,1.019782e+17,99566408358330419
2,ESPORTSTMNT01:1110177,207700.0,100.0,836608,200,1546286.0,200.0,100.0,100783238110621058,1.019782e+17,99566408358330419
3,ESPORTSTMNT01:1110187,359245.0,200.0,626524,100,1628728.0,200.0,100.0,98767991949608898,1.019782e+17,99566406336293850
4,ESPORTSTMNT01:1110198,148483.0,100.0,895424,200,1694081.0,200.0,100.0,99566406336293850,1.019782e+17,99566408358330419
